In [1]:
# Changing the the directory where all modules are saved.
import os
os.chdir(r'C:\Users\vladi\OneDrive\Desktop\Sigmoid\CV\Object detection\object detection\object detection\utils')

In [2]:
# Importing all needed libraries.
import numpy as np
import torch
import torch.utils.data
from PIL import Image
import pandas as pd

from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
import torchvision

from engine import train_one_epoch, evaluate
import utils
import transforms as T
from PIL.ExifTags import TAGS, GPSTAGS

c:\python38\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
c:\python38\lib\site-packages\numpy\.libs\libopenblas.PYQHXLVVQ7VESDPUVUADXEVJOBGHJPAY.gfortran-win_amd64.dll
c:\python38\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [3]:
def get_model(num_classes):
    '''
        This function is used to create a pretrained FastRCNN Model.
    :num_classes: int
        The number of classes. Should be setted n+1 wher n is the number of object to detect.
    '''
    model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
    
    #get the number of input features for the classifier
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    
    # replace the pre-trained head with a new one
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)
    
    return model

In [4]:
def get_transform(train = False):
    '''
        Uses to create a transformer.
    :param train: bool
        Decides if to add or not the RandomHorizontalFlip transformation.
    '''
    transforms = []
    # convert the image, a PIL image, into a PyTorch Tensor
    transforms.append(T.ToTensor())
    if train:
        # during the training, randomly flip the trinaing images
        # and ground-truth for data augmentation
        transforms.append(T.RandomHorizontalFlip(0.5))
    return T.Compose(transforms)

In [5]:
transform = get_transform()

['1632492310200.jpg',
 '1632492310204.jpg',
 '1632492310206.jpg',
 '1632492310208.jpg',
 '1632492310209.jpg',
 '1632492310211.jpg',
 '1632492366195.jpg',
 '1632492366197.jpg',
 '1632492366199.jpg',
 '1632492366201.jpg',
 '1632492366204.jpg',
 '1632492412542.jpg',
 '1632492412545.jpg',
 '900px-Times_Square_Panorama.jpg',
 'newFile-3.jpg',
 'TimesSquare_bright.jpg']

In [17]:
os.chdir(r'C:\Users\vladi\OneDrive\Desktop\Sigmoid\CV\Logo bluring\images_test')
path_dir = os.getcwd()
paths = os.listdir()

#paths = [r'C:\Users\vladi\OneDrive\Desktop\Sigmoid\CV\Logo bluring\images_test\900px-Times_Square_Panorama.jpg',r'C:\Users\vladi\OneDrive\Desktop\Sigmoid\CV\Logo bluring\images_test\newFile-3.jpg',r'C:\Users\vladi\OneDrive\Desktop\Sigmoid\CV\Logo bluring\images_test\TimesSquare_bright.jpg']
for path in paths:
    img_path = os.path.join(path_dir,path)
    img = Image.open(img_path).convert('RGB')
    # Readding the image metadata.
    for orientation in TAGS.keys() : 
        if TAGS[orientation]=='Orientation' : break 
    # Getting the exif
    exif=dict(img.getexif().items())
    # Rotating the image if the orientation is wrong.
    #         print(img)
    #         print(img.getexif())
    #print(TAGS.keys())
    if len(exif)!=0:
        if orientation in exif.keys():
            if   exif[orientation] == 3 : 
                img=img.rotate(180, expand=True)
            elif exif[orientation] == 6 : 
                img=img.rotate(270, expand=True)
            elif exif[orientation] == 8 : 
                img=img.rotate(90, expand=True)
                
    img = T.Compose([T.ToTensor()])(img,'0')
    imgs.append(img)

In [18]:
loaded_model = get_model(num_classes=2)
loaded_model.load_state_dict(torch.load(r'C:\Users\vladi\OneDrive\Desktop\Sigmoid\CV\Logo bluring\logo_detection_model\fix1_ports_model.pth'))

<All keys matched successfully>

In [1]:
from PIL import ImageDraw
# Getting the image.
for idx in range(len(imgs)):
    img, _ = imgs[idx]
    # Getting the object coordinates.
#     label_boxes = np.array(test[idx][1]['boxes'])

    # Setting the model to eval state.
    loaded_model.eval()
    # Making the prediction.
    with torch.no_grad():
        prediction = loaded_model([img])

    # Getting an drawing the image.
    image = Image.fromarray(img.mul(255).permute(1, 2, 0).byte().numpy())
    draw = ImageDraw.Draw(image)

    # Drawing the real box around the object.
#     for elem in range(len(label_boxes)):
#         draw.rectangle([(label_boxes[elem][0], label_boxes[elem][1]),
#                        (label_boxes[elem][2], label_boxes[elem][3])],
#                       outline='green', width=3)
    # Drawing the predicted box around the object.
    for element in range(len(prediction[0]['boxes'])):
        boxes = prediction[0]['boxes'][element].cpu().numpy()
        score = np.round(prediction[0]['scores'][element].cpu().numpy(), decimals=4)

        if score > 0.3:
            draw.rectangle([(boxes[0], boxes[1]), (boxes[2], boxes[3])],
                         outline='red', width=3)
            draw.text((boxes[0], boxes[1]), text=str(score))
    display(image)